In [9]:
Rw.<w> = PowerSeriesRing(QQ, default_prec=11)
Ru.<u> = PowerSeriesRing(Rw, default_prec=2400) # default_prec should be 4x the desired output precision

def B(X, cutoff):
    Y = (1/X).power_series() +O(u^cutoff)
    Y2 = Y^2 + O(u^cutoff)
    accu = bernoulli(2) * Y  / 2 
    YPow = Y
    for k in range(4, cutoff):
        if k%2==0:
            YPow = YPow * Y2 + O(u^cutoff)
            accu = accu + bernoulli(k) * YPow / (k*(k-1))
    return accu
    #return sum( bernoulli(k) * Y^(k-1)  / (k*(k-1)) for k in range(2, cutoff) if k%2 ==0 ) +O(u^cutoff)
def El(l):
    return sum(moebius(l/d)*u^(-d) for d in divisors(l) )/l
def lal(l):
    return u^l*(1-u^l)*l

def wt11Xl(l):
    return sum(moebius(l/d)*(1-w^d+O(w^11)) for d in divisors(l))/l

def logUl1(X, l, cutoff):
    return ( X*(log( (lal(l)*El(l)).power_series() +O(u^cutoff)) -1) +(-El(l)+X-1/2)*log((1-(X +O(u^(l+cutoff)) )/El(l)).power_series())).power_series()
def logUl2(X, l, cutoff):
    return B(-El(l)+X, cutoff)-B(-El(l), cutoff) 
def logUl2a(X, l, cutoff):
    return B(-El(l)+X, cutoff)
def logUlp1(l, cutoff):
    return logUl1(wt11Xl(l),l,cutoff)
def logUlp2(l, cutoff):
    return logUl2(wt11Xl(l),l,cutoff)
def logUlp2a(l, cutoff):
    return logUl2a(wt11Xl(l),l,cutoff)

def logUl(X, l, cutoff):
    return logUl1(X, l, cutoff) + logUl2(X, l, cutoff)
    #return ( X*(log( (lal(l)*El(l)).power_series() ) -1) +(-El(l)+X-1/2)*log((1-X/El(l)).power_series())).power_series()+B(-El(l)+X, cutoff)-B(-El(l), cutoff) 
def logUlp(l, cutoff):
    return logUlp1(l, cutoff) + logUlp2(l, cutoff)

def mindeg(X):
    return min([99999] + X.exponents())


In [13]:
def preZ(cutoff):
    # need to go to twice the cutoff
    #return -u*(exp(sum(logUlp(l,cutoff) for l in range(1, 2*cutoff) ))-1)
    return -u*(exp(sum(logUlp1(l,cutoff) for l in range(1, 2*cutoff) ) + sum(logUlp2(l,cutoff) for l in range(1, cutoff/2) ))-1)
def makeZ(Z):
    return Z.map_coefficients(lambda c: (c.truncate(11))(1) )

def pickwcoeff(Z,j):
    return Z.map_coefficients( lambda c: c[j])
def bigZ(cutoff):
    return makeZ(preZ(cutoff))
def preZl(l, cutoff):
    return  exp(logUlp(l,cutoff))
def preZl2(l, cutoff):
    return  exp(logUlp2(l,cutoff))

# The asymptotic formula for the Euler characteristic
def asymp_Z(g):
    Cev = 1/factorial(10) * 1024 * pi^2 *(14175 - 4725 * pi^2 + 630 * pi^4 - 45 * pi^6 + 2 * pi^8)
    Codd = 4* pi / factorial(10) * 1280 *(2835 + 2 * pi^2 * (-945 + 189 * pi^2 - 18 * pi^4 + pi^6))
    return factorial(g-2) / (2*pi)^g * ((-1)^(g/2) * Cev if g%2==0 else (-1)^((g-1)/2)*Codd)

In [15]:
# code to compute the leading oder terms of the series \mathbb{A}
def computeAA(cutoff):
    return exp(sum(logUlp1(l,cutoff) for l in range(1, 2*cutoff) ) + sum(logUlp2(l,cutoff) for l in range(2, cutoff/2) ))-1

The following commands compute the series Z up to various cutoffs

In [181]:
%time theZ_30 = bigZ(30)

CPU times: user 18.1 s, sys: 26.9 ms, total: 18.1 s
Wall time: 18.1 s


In [183]:
%time theZ_40 = bigZ(40)

CPU times: user 27 s, sys: 80.2 ms, total: 27 s
Wall time: 27.1 s


In [201]:
%time theZ_70 = bigZ(70)

CPU times: user 1min 8s, sys: 509 ms, total: 1min 9s
Wall time: 1min 9s


In [205]:
%time theZ_200 = bigZ(200)

CPU times: user 14min 47s, sys: 3 s, total: 14min 50s
Wall time: 14min 52s


In [209]:
%time theZ_400 = bigZ(400)

CPU times: user 1h 57min 45s, sys: 32.2 s, total: 1h 58min 18s
Wall time: 1h 58min 28s


In [323]:
%time theZ_600 = bigZ(600)

CPU times: user 6h 39min 10s, sys: 1min 45s, total: 6h 40min 55s
Wall time: 7h 52min 7s


In [8]:
#save(theZ_400, "data/theZ_400")
#theZ_400=load("data/theZ_400.sobj")

In [ ]:
lppp=list_plot([(g, (theZ_600[g][0]/asympZ(g)).n(digits=10) ) for g in range(30,600) ])
lppp.save_image("data/ECrelplot600.png")
lppp

In [16]:
# computation to go into the mathematica file
%time AA70 = computeAA(70)

CPU times: user 4min 3s, sys: 217 ms, total: 4min 3s
Wall time: 4min 4s


In [17]:
AA

(-1 + w + O(w^11))*u + (1/12 - 13/12*w + w^3 + O(w^11))*u^2 + (w - w^2 - w^3 + w^4 + O(w^11))*u^3 + (-7/1440 - 779/720*w + 601/288*w^2 - 11/12*w^3 - 13/12*w^4 + w^6 + O(w^11))*u^4 + (-7/1440 + 161/160*w - 479/480*w^2 - 1/288*w^3 + 2*w^4 - 2*w^5 - w^6 + w^7 + O(w^11))*u^5 + (-1549/362880 - 249407/120960*w + 55913/17280*w^2 - 13541/51840*w^3 - 4313/1440*w^4 + 1177/288*w^5 - 23/12*w^6 - 13/12*w^7 + w^9 + O(w^11))*u^6 + (-667/181440 + 92107/45360*w - 11743/3024*w^2 + 5177/3240*w^3 + 37067/25920*w^4 - 901/720*w^5 + 23/288*w^6 + 2*w^7 - 2*w^8 - w^9 + w^10 + O(w^11))*u^7 + (-39503/12441600 - 46404997/21772800*w + 64998739/14515200*w^2 - 7545557/4354560*w^3 - 76969741/17418240*w^4 + 8483/1440*w^5 - 60881/51840*w^6 - 4313/1440*w^7 + 1177/288*w^8 - 23/12*w^9 - 13/12*w^10 + O(w^11))*u^8 + (-80281/29030400 + 29810213/29030400*w - 7889981/1612800*w^2 + 92003533/14515200*w^3 + 24286327/5806080*w^4 - 849683/71680*w^5 + 52049/6480*w^6 - 68953/25920*w^7 + 539/720*w^8 - 265/288*w^9 + 2*w^10 + O(w^11))*u